In [ ]:
# TYPING IN PYTHON
![视频链接]()

## 类型注解放在什么地方
变量后面或者函数后面
### 变量类型
- int
- float
- str
- bool
- bytes
- Any
- Tuple
- List, list[dtype]
- Dist, dict[type(key),type(value)]

既可以是Python的基础类型，也可以是未实例化的类或者是typing里面提供的泛型

In [ ]:
def f(a: int, b: int) -> int:
    return a+b


## 使用自己的类进行注解
### 注意循环引入的问题
解决方式：
- TYPE_CHECKING: 有时候没啥用
```python
if TYPE_CHECKING:
    from student import student
```
- 在一个注解的类型名称外面加''
```python
def get_student_name(self, s:'Student')->str:
    return s.get_student_name()
```
- import annotations 仓库
其实是自动在循环引用的类型外部加引号
```python
from __future__ import annotations
```

In [6]:
from __future__ import annotations

class Teacher:
    def get_name(self)->str:
        return "sch"
    
    # 出现了循环引入的问题
    # 1. 在一个类的外面加引号
    # def get_student_name(self, s: 'Student') -> str:
    def get_student_name(self, s: Student) -> str:
        return s.get_name()
    
class Student:
    def get_name(self)->str:
        return "zhaoyue"
    
    # 标注自定义类帮助获取自定义类中函数所标注的typing信息
    def get_teacher_name(self,teacher: Teacher)->str:
        return teacher.get_name()

In [3]:
s=Student()
t=Teacher()

print(s.get_name())
print(s.get_teacher_name(t))

zhaoyue
sch


## final不能修改的注解器


python中没有常量这一概念，但可以限定某些变量不能被修改

Final: 设置变量不能被修改
@final: 装饰器，设置父类 函数不能被子类函数覆盖

In [2]:
from typing import Final, final

a:Final[int]=10

class Human:
    
    @final
    def get_name(self)->str:
        return ""

11


## 类型注解选项
- Optional：在变量初始化的时候
Optional[Teacher]: 既可以是Teacher， 也可以是None
也可以：Teacher | None
- Union: 参数中
Union[str, int]: 函数参数既可以是str也可以是int
也可以：str | int

更精确的定义函数的类型 

## 特殊类型
- Self：返回类自身,return this 的情形
有助于链式调用

- ClassVar：相当于static

In [33]:
from __future__ import annotations
from typing_extensions import Self
from typing import ClassVar

class Number:
    
    __op_num: ClassVar[int]=0
    
    def __init__(self, number: int)-> None:
        self._number=number
        
    # 将对象本身返回出去
    # also: def double(self)->'Number': but cause trouble when changing class name
    def double(self)->Self:
        self._number*=2
        Number.__op_num+=1

        return self
    def add_one(self)->Self:
        self._number+=1
        Number.__op_num+=1
        return self
    def __str__(self) -> str:
        return f"Current number is {self._number}"
    
n: Number=Number(10)
n1: Number=Number(13)

print(n.double().add_one())
print(Number.__op_num)
print(n.__op_num)
print(n1.__op_num)

Current number is 21


AttributeError: type object 'Number' has no attribute '__op_num'